In [2]:
import ants
import nibabel as nib
import numpy as np

In [46]:
avgt = nib.load("/Users/mahdi/stage-2022-mahdi/data/AVGT.nii.gz")
avgt_data = avgt.get_fdata()
avgt_affine = avgt.affine
avgt_shape = avgt.shape

## registration

In [47]:
ls = nib.load("/Users/mahdi/Desktop/test_ls/lightsheetRAS.nii")

In [48]:
fvol = avgt_data.astype(np.float32)
mvol = ls.get_fdata().astype(np.float32)

In [49]:
fi = ants.from_numpy(fvol).resample_image(avgt_shape, 1, 0)
mi = ants.from_numpy(mvol).resample_image(avgt_shape, 1, 0)

In [54]:
mytx = ants.registration(
    fixed=fi,
    moving=mi,
    type_of_transform='AffineFast'
)

In [55]:
wmi = ants.apply_transforms(
    fixed=fi,
    moving=mi,
    transformlist=mytx['fwdtransforms']
)

In [57]:
aligned = nib.Nifti1Image(wmi.numpy(),avgt_affine)

In [58]:
nib.save(aligned,"/Users/mahdi/Desktop/test_ls/test_reg/aligned1.nii")

## ASR to RAS

In [9]:
lightsheet = nib.load("/Users/mahdi/Desktop/mice01-2xy.nii")

In [39]:
data = lightsheet.get_fdata()
aff = lightsheet.affine

In [40]:
def switch_axis(vol):
    vol = np.moveaxis(vol, (0, 1, 2), (2, 0, 1))
    vol = np.flip(vol, axis=2)
    vol = np.flip(vol, axis=1)
    return vol

In [41]:
print(data.shape)

(1667, 2679, 660)


In [42]:
data = switch_axis(data)

In [43]:
print(data.shape)

(2679, 660, 1667)


In [45]:
dimg = nib.Nifti1Image(data, aff)
nib.save(dimg, "/Users/mahdi/Desktop/test_ls/lsRAS.nii")